# Create azure resorces from template

It is always possible to create Azure resources using the UI. However, for automation it is better to use templates in which the deployment of resources is defined.

In this directory the template and parameters for automated deployment are given. However, the parameters need to be individually adapted to your azure account (subscription id, resource group name, location, etc.)

In [25]:
# Before you start make sure you're logged into your Azure account
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "1fdb6d07-480b-4c31-a913-e847d51e7446",
    "id": "ea54fca2-3a3e-4b3b-91e7-a7bf971e0443",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "1fdb6d07-480b-4c31-a913-e847d51e7446",
    "user": {
      "name": "Philliplakaschus@gmail.com",
      "type": "user"
    }
  }
]


## Deploy resources from template

https://learn.microsoft.com/en-us/azure/azure-resource-manager/templates/deploy-python#deploy-local-template

In [26]:
import os
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
from dotenv import load_dotenv
load_dotenv()

# Get resource client
credential = AzureCliCredential()
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
resource_client = ResourceManagementClient(credential, subscription_id)

In [27]:
# Load parameters and template
import json

with open("parameters.json", "r") as template_file:
    params = json.load(template_file)['parameters']

with open("template.json", "r") as template_file:
    template_body = json.load(template_file)

In [28]:
# Make sure the resource group exists
rg_name = params['resourceGroupName']['value']
rg_location = params['location']['value']
rg_result = resource_client.resource_groups.create_or_update(
    rg_name,
    {
        "location": rg_location
    }
)

print(f"Provisioned resource group with ID: {rg_result.id}")

Provisioned resource group with ID: /subscriptions/ea54fca2-3a3e-4b3b-91e7-a7bf971e0443/resourceGroups/ai102-example


In [29]:
from azure.mgmt.resource.resources.models import DeploymentMode

deployment_name = params['name']['value']
rg_deployment_result = resource_client.deployments.begin_create_or_update(
    resource_group_name=rg_name,
    deployment_name=deployment_name,
    parameters={
        "properties": {
            "template": template_body,
            "parameters": params,
            'mode': DeploymentMode.incremental
        }
    }
)

In [31]:
rg_deployment_result.status()

'Succeeded'

In [32]:
# Get the endpoint of the deployed web app
ai_endpoint_name = f"https://{deployment_name}.cognitiveservices.azure.com/"

In [35]:
# Get the key of the deployed web app
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

client = CognitiveServicesManagementClient(credential, subscription_id)

# Get the account keys
keys = client.accounts.list_keys(rg_name, deployment_name)

# For safety never print out the keys
ai_key = keys.key1


Note that the deployment take a few minutes. After successful deployment continue testing the Azure AI service resource. There are two ways to access the AI service endpoint. Note that each call costs a small amount of money.

## Option 1: Use REST client

In [36]:
import os
import http.client, json

In [37]:
def main():
    global ai_endpoint
    global ai_key

    try:
        # Get Configuration Settings
        load_dotenv()
        ai_endpoint = ai_endpoint_name
        ai_key = ai_key

        # Get user input (until they enter "quit")
        userText =''
        while userText.lower() != 'quit':
            userText = input('Enter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                GetLanguage(userText)


    except Exception as ex:
        print(ex)

def GetLanguage(text):
    try:
        # Construct the JSON request body (a collection of documents, each with an ID and text)
        jsonBody = {
            "documents":[
                {"id": 1,
                 "text": text}
            ]
        }

        # Let's take a look at the JSON we'll send to the service
        print(json.dumps(jsonBody, indent=2))

        # Make an HTTP request to the REST interface
        uri = ai_endpoint.rstrip('/').replace('https://', '')
        conn = http.client.HTTPSConnection(uri)

        # Add the authentication key to the request header
        headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': ai_key
        }

        # Use the Text Analytics language API
        conn.request("POST", "/text/analytics/v3.1/languages?", str(jsonBody).encode('utf-8'), headers)

        # Send the request
        response = conn.getresponse()
        data = response.read().decode("UTF-8")

        # If the call was successful, get the response
        if response.status == 200:

            # Display the JSON response in full (just so we can see it)
            results = json.loads(data)
            print(json.dumps(results, indent=2))

            # Extract the detected language name for each document
            for document in results["documents"]:
                print("\nLanguage:", document["detectedLanguage"]["name"])

        else:
            # Something went wrong, write the whole response
            print(data)

        conn.close()


    except Exception as ex:
        print(ex)

In [38]:
main()

{
  "documents": [
    {
      "id": 1,
      "text": "present"
    }
  ]
}
{
  "documents": [
    {
      "id": "1",
      "detectedLanguage": {
        "name": "English",
        "iso6391Name": "en",
        "confidenceScore": 1.0
      },
      "warnings": []
    }
  ],
  "errors": [],
  "modelVersion": "2023-12-01"
}

Language: English


## Option 2: Use Python SDK

In [39]:
from dotenv import load_dotenv
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [40]:
def main():
    global ai_endpoint
    global ai_key

    try:
        # Get Configuration Settings
        load_dotenv()
        ai_endpoint = ai_endpoint_name
        ai_key = ai_key

        # Get user input (until they enter "quit")
        userText =''
        while userText.lower() != 'quit':
            userText = input('\nEnter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                language = GetLanguage(userText)
                print('Language:', language)

    except Exception as ex:
        print(ex)

def GetLanguage(text):

    # Create client using endpoint and key
    credential = AzureKeyCredential(ai_key)
    client = TextAnalyticsClient(endpoint=ai_endpoint, credential=credential)

    # Call the service to get the detected language
    detectedLanguage = client.detect_language(documents = [text])[0]
    return detectedLanguage.primary_language.name

In [41]:
main()

Language: English


## Optional: Delete resource group

Since every information for the deployment is contained in the template, you can now also delete the entire resource group to make sure you're not getting charged for anything.

In [14]:
# Properly delete the resource group
cleanup_operation = resource_client.resource_groups.begin_delete(rg_name)

In [76]:
cleanup_operation.status()

'Succeeded'